<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-10-24 04:02:41--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-10-24 04:02:42 (33.3 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [3]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-10-24 04:02:50--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.207, 108.177.121.207, 142.251.183.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.02s   

2025-10-24 04:02:51 (37.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [12]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 1 
  (real disaster)
Text : 
 A brief violent storm swept through the Chicago area Sunday afternoon leading to one death and an evacuation of Lollapalooza and more
---

Target : 0 
  not real disaster
Text : 
 Owner of Chicago-Area Gay Bar Admits to Arson Scheme: Frank Elliott pleaded guilty to hiring an arsonist to to... http://t.co/jCFEhrHLq8
---

Target : 1 
  (real disaster)
Text : 
 Senators calling for emergency housing: Boxer Feinstein back plan to move #homeless vets to VA campus http://t.co/Gm80X3vutf
---

Target : 0 
  not real disaster
Text : 
 @DmoneyDemi I had my meltdown yesterday.  I'm going to miss you so much. You are forsure my DTB for life. When I get back watchout ??
---

Target : 1 
  (real disaster)
Text : 
 .@Vagersedolla visits villages recently bombed by Turkey and finds people fed up with the PKK http://t.co/UUWEiKD7sP
---



In [13]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [14]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [16]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [17]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [18]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 (OFFICIAL VID) &gt; #DoubleCups &gt;&gt; https://t.co/lfKMTZaEkk &gt;&gt; @TrubGME Prod @THISIZBWRIGHT &gt;&gt; #ARMAGEDDON                 . 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [19]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [20]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [21]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 Look: #I have collapsed #after attempting to munch an endangered species.



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[ 4.6506491e-02,  1.4164988e-02, -1.0480881e-03, ...,
          3.3488993e-02, -1.6133558e-02, -1.3996802e-02],
        [ 4.6285477e-02,  5.7812557e-03, -3.2888256e-02, ...,
         -5.6624413e-06,  3.8006220e-02, -3.7045132e-02],
        [ 9.0920329e-03, -4.0367387e-02,  7.9524145e-03, ...,
          3.5651494e-02, -4.9711801e-02,  2.9194761e-02],
        ...,
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03],
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03],
        [ 4.1544262e-02,  3.8823750e-02, -4.3369643e-03, ...,
          3.6131728e-02, -4.8943687e-02, -9.9865906e-03]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [23]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [33]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_true,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,y_pred=y_pred,average='weighted')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results

In [34]:
# Get Baseline Results
baseline_results=calculate_results(y_true=val_labels,y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# **Model1 : Feed Forward Neural Network ( Dense Model)**

In [37]:
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers

In [40]:
SAVE_DIR="ModelLogs"

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # Add this layer to reduce dimensions
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_Dense")
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_1_history=model_1.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: ModelLogs/model_1_dense/20251024-064543
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5557 - loss: 0.6840 - val_accuracy: 0.5433 - val_loss: 0.6963
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5842 - loss: 0.6697 - val_accuracy: 0.6299 - val_loss: 0.6598
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6219 - loss: 0.6503 - val_accuracy: 0.6168 - val_loss: 0.6444
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6439 - loss: 0.6341 - val_accuracy: 0.6640 - val_loss: 0.6198
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6753 - loss: 0.6129 - val_accuracy: 0.6509 - val_loss: 0.6237


In [42]:
model_1.evaluate(val_sentances,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6464 - loss: 0.6462


[0.623660683631897, 0.6509186625480652]

In [43]:
model_1_predict_probs=model_1.predict(val_sentances)
model_1_predict_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[0.441873  ],
       [0.56637543],
       [0.62882084],
       [0.25346127],
       [0.347933  ],
       [0.5374462 ],
       [0.61757904],
       [0.5580428 ],
       [0.5725021 ],
       [0.20823337],
       [0.38588038],
       [0.43684903],
       [0.47977015],
       [0.36418056],
       [0.5252877 ],
       [0.39477822],
       [0.20861995],
       [0.29424295],
       [0.3260463 ],
       [0.4161183 ],
       [0.19044794],
       [0.10061562],
       [0.30691427],
       [0.36289054],
       [0.4029231 ],
       [0.5134693 ],
       [0.10110677],
       [0.43567425],
       [0.17998363],
       [0.22076803],
       [0.3549491 ],
       [0.36171046],
       [0.4391593 ],
       [0.3682958 ],
       [0.4338756 ],
       [0.11391601],
       [0.54043925],
       [0.20278646],
       [0.18965393],
       [0.5870159 ],
       [0.28281024],
       [0.50212675],
       [0.3081129 ],
       [0.41848502],
       [0.27322412],
       [0.4875286 ],
       [0.35142255],
       [0.657

In [44]:
model_1_predict_probs.shape

(762, 1)

In [45]:
model_1_predict_probs[0]

array([0.441873], dtype=float32)

In [47]:
# Make the preictions comaparable with the actaul labels
model_1_predict_probs=tf.squeeze(tf.round(model_1_predict_probs))
model_1_predict_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

# ***Calculate Model Results***

In [50]:
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_predict_probs)
model_1_results

{'accuracy': 0.6509186351706037,
 'precsion': 0.6814052089642642,
 'recall': 0.6509186351706037,
 'f1_score': 0.6199277469577666}

In [53]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

# ***Lets Visualize the embedding that Model learned using Tensorflow Projector***

In [54]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(100,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [55]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,789 (151.52 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,860 (101.02 KB)

In [58]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights,print(embed_weights.shape)

(100, 128)


(array([[-0.02001623,  0.03515048,  0.02171817, ..., -0.01733577,
          0.01981488, -0.03238846],
        [ 0.2107572 , -0.1618422 , -0.16945435, ...,  0.1960888 ,
         -0.19072764,  0.15263574],
        [-0.1843201 ,  0.11794026,  0.16367866, ..., -0.14328706,
          0.1824277 , -0.1899749 ],
        ...,
        [ 0.01247601,  0.00373714, -0.03959863, ...,  0.05834554,
         -0.05785266,  0.03516963],
        [-0.42073542,  0.32182497,  0.4227325 , ..., -0.3164959 ,
          0.39596078, -0.32230723],
        [-0.28698093,  0.2727575 ,  0.35410658, ..., -0.24189903,
          0.3330483 , -0.3176406 ]], dtype=float32),
 None)

Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).  This code is from the tensorflow documentation Here[https://www.tensorflow.org/text/guide/word_embeddings]

In [59]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [60]:
# After this download step use projector tensorflow to visualzize it
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>